What does this notebook do:

Inputs: 
- request_name
- filters: keywords, dates, sources / stance (optional)
- narratives (to check vs news via NLI)

Outputs: 
- topics df's for dashboard: 'df_ner', 'df_narr', 'df_cluster_names', 'df_text_table', 'df_cross'

Default DB - df_war_ukr_filtered.csv - war in Ukraine, from feb'22 till jan'23, embeddings - mnet, n_dim = 384 (default for bertopic).

# Colab Forms

In [2]:
keywords1 = "войн, воен"
keywords1.split(',')

['войн', ' воен']

In [18]:
# @title User defined filters

# @markdown Lists of keywords, separated by comma (","). Within list - OR, between lists - AND
keywords1 = "" # @param {type:"string"}
keywords2 = "" # @param {type:"string"}
keywords3 = "" # @param {type:"string"}
keywords = [keywords1.split(','), keywords2.split(','), keywords3.split(',')]

#@markdown ---

# @markdown Period search. If finish_date skipped - filter till last possible date.
start_date = '2022-08-01' # @param {type:"date"}
finish_date = '' # @param {type:"date"}
if finish_date: dates = [start_date, finish_date]
else: dates = [start_date]

# @markdown Source(s). Must be exact match to the source name in the dataset. Separated by comma.
sources = "" # @param {type:"string"}
sources = sources.split()
# @markdown List of content types. Must be exact match to the content type in the dataset. Separated by comma.
content_type = "" # @param {type:"string"}
content_type = content_type.split()

# @markdown Narratives for NLI.

# @markdown (Attention! ~1sec per each 'narrative vs news' pair)
narr_list = ['ВСУ отступают', 'Бойцы Вагнер совершают преступления'] # @param {type:"string"}
# Threshold for NLI
threshold_nli = 0.7 # @param {type:"slider", min:0, max:1, step:0.1}

#@markdown ---

# @markdown Folder name (without '/') for backup
# @markdown (for dashboard the results are overwritten each time)
request_name = 'wagner' #@param {type:"string"}

# set user defined filters & params

In [ ]:
# User defined filters
keywords = [['Вагнер']] # list of lists of keywords. Within list - OR, between lists - AND
dates = ['2022-08-01'] # list of min and max dates. If only one date - assumed to be min date.
sources = [] # list of sources. Must be exact match to the source name in the dataset
content_type = [] # list of content types. Must be exact match to the content type in the dataset

# Narratives for NLI
# Attention! ~1sec per each 'narrative vs news' pair
narr_list = ['ВСУ отступают', 'Бойцы Вагнер совершают преступления']
# Threshold for NLI
threshold_nli = 0.5
# folder name (without '/') for backup (for dashboard results are overwritten each time)
request_name = 'wagner'

In [ ]:
# User defined filters
keywords = [] # list of lists of keywords. Within list - OR, between lists - AND
dates = ['2023-01-10'] # list of min and max dates. If only one date - assumed to be min date.
sources = [] # list of sources. Must be exact match to the source name in the dataset
content_type = [] # list of content types. Must be exact match to the content type in the dataset

# Narratives for NLI
# Attention! ~1sec per each 'narrative vs news' pair
narr_list = ['ВСУ отступают', 'Российские войска не атакуют мирное население', 'Украина атакует мирное население', 'На Украине создают биологическое оружие']
# Threshold for NLI
threshold_nli = 0.5
# folder name (without '/') for backup (for dashboard results are overwritten each time)
request_name = 'last_3_weeks_no_filters_3narrs'

add:
- choose kmeans vs hdbscan
- basic output info: filters, N of news, N of clusters (-1 size), v measure

# check env - install respective libs & set paths

In [19]:
# Function to check env (colab or local)
import sys

def check_environment():
    if 'google.colab' in sys.modules:
        # Running in Google Colab
        return "Google Colab"

    if hasattr(sys, 'prefix'):
        if sys.prefix.startswith('/usr/local'):
            # Running in a Colab-like environment (could be Colab or another cloud environment)
            return "Colab-like environment"
        else:
            # Running in a local environment
            return "Local environment"

    # Default to local environment if checks are inconclusive
    return "Local environment"

In [20]:
# set paths and install packages depending on the environment

if check_environment() == 'Google Colab':
    !pip install transformers torch bertopic openai natasha --quiet
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/TG news channels/'
    result_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/TG news channels/df_4_dashboard/'

else:
    data_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/TG news channels/'
    result_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/TG news channels/df_4_dashboard/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 82.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (set

In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import unicodedata

# Load pre-processed data & filter (keywords, dates, sources)

Pre-processed = what can be calculated after loading news:
- NLI narratives

In [22]:
df_full = pd.read_csv(data_path+'filtered/df_war_ukr_filtered.csv')
df_full.shape

<ipython-input-22-ac4e43e1873c>:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(data_path+'filtered/df_war_ukr_filtered.csv')


(114809, 398)

In [23]:
df_full.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114809 entries, 0 to 114808
Columns: 398 entries, channel_name to 383
dtypes: bool(2), float64(384), int64(4), object(8)
memory usage: 700.9 MB


#### deduplicate, basic info

In [ ]:
# texts deduplication
df_full.drop_duplicates(subset=['channel_name', 'msg_id'], inplace=True)

# texts cleaning from emojis, links and cut to 750 symbols
def remove_emojis_and_links(text):
    # Unicode range for emojis
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # Emoticons
                               "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                               "]+", flags=re.UNICODE)

    # Regular expression for URLs
    url_pattern = re.compile(r"http\S+|www\S+")

    # Remove emojis
    text = emoji_pattern.sub(r'', text)

    # Remove URLs
    text = url_pattern.sub(r'', text)

    # Remove any remaining variation selectors
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')

    return text
df_full['cleaned_message'] = df_full['cleaned_message'].apply(lambda x: remove_emojis_and_links(x))
# # cut cleaned_message to 750 symbols
df_full['cleaned_message'] = df_full['cleaned_message'].apply(lambda x: x[:750])

In [ ]:
df_full.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114809 entries, 0 to 114808
Columns: 398 entries, channel_name to 383
dtypes: bool(2), float64(384), int64(4), object(8)
memory usage: 700.9 MB


In [ ]:
# convert str into datetime
df_full['date'] = pd.to_datetime(df_full['date'], format='%Y-%m-%d')
df_full.date.min(), df_full.date.max()

(Timestamp('2022-02-01 00:00:00'), Timestamp('2023-01-31 00:00:00'))

In [ ]:
df_full.columns[:20]

Index(['channel_name', 'msg_id', 'message', 'cleaned_message', 'date', 'views',
       'number_replies', 'number_forwards', 'contains_media', 'media_type',
       'has_url', 'url', 'domain', 'document_type', '0', '1', '2', '3', '4',
       '5'],
      dtype='object')

## Filtering by keywords, dates, sources

In [24]:
# Filtered df as per user defined filters
df_filtered = df_full.copy()
if keywords[0]:
    for keyword_list in keywords:
        df_filtered = df_filtered[df_filtered['cleaned_message'].str.contains('|'.join(keyword_list), case=False)]
if dates:
    if len(dates) == 1:
        df_filtered = df_filtered[df_filtered['date'] >= dates[0]]
    elif len(dates) == 2:
        df_filtered = df_filtered[(df_filtered['date'] >= dates[0]) & (df_filtered['date'] <= dates[1])]
if sources:
    df_filtered = df_filtered[df_filtered['source'].isin(sources)]
if content_type:
    df_filtered = df_filtered[df_filtered['content_type'].isin(content_type)]

print(f"Number of messages in the filtered dataset: {df_filtered.shape[0]}, {df_filtered.shape[0]/df_full.shape[0]*100:.2f}% of the full dataset")

Number of messages in the filtered dataset: 49005, 42.68% of the full dataset


# Share of filtered from total news

В analyze_all_sources.ipynb ф-ция для поиска в многих csv-файлах (по источникам), фильтрации и созд-я df и df_cross.

In [ ]:
# filter df_full by same dates to keep the same base
if dates:
    if len(dates) == 1:
      df_full = df_full[df_full.date >= dates[0]]
    elif len(dates) == 2:
      df_full = df_full[(df_full.date >= dates[0]) & (df_full.date >= dates[1])]

# calculate crosstab
df_cross_all = df_full.groupby(['date', 'channel_name']).agg({'cleaned_message': 'count', 'views': 'sum'}).reset_index()
df_cross_filtered = df_filtered.groupby(['date', 'channel_name']).agg({'cleaned_message': 'count', 'views': 'sum'}).reset_index()
# combine df_cross_all and df_cross_filtered naming common columns respectively and calculate share of filtered messages
df_cross = pd.merge(df_cross_all, df_cross_filtered, on=['date', 'channel_name'], how='left', suffixes=('_all', '_filtered')).fillna(0)

In [ ]:
# add to df_cross types from csv-file sources_types.csv
# try open csv with comma delimiter; if it doesn't work - try with semicolon
cols = ['channel_name', 'channel_code', 'media_type', 'content_type', 'audience size']
try:
    df_types = pd.read_csv(data_path+'channels.csv', sep=',')
    df_cross = pd.merge(df_cross, df_types[cols], on='channel_name', how='outer')
except:
    df_types = pd.read_csv(data_path+'channels.csv', sep=';')
    df_cross = pd.merge(df_cross, df_types[cols], on='channel_name', how='outer')
# df_types = pd.read_csv(data_path+'channels.csv')
# df_cross = pd.merge(df_cross, df_types, on='channel_name', how='outer')

In [ ]:
# percentage of filtered messages
df_cross['share_filtered'] = df_cross['cleaned_message_filtered'] / df_cross['cleaned_message_all']
# percentage of filtered views
df_cross['share_views_filtered'] = df_cross['views_filtered'] / df_cross['views_all']

# NER & Narratives

## Add NER to dataset

If not pre-processed

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [ ]:
def get_entities(text):
     doc = Doc(text)
     doc.segment(segmenter)
     doc.tag_morph(morph_tagger)
     for token in doc.tokens:
          token.lemmatize(morph_vocab)
     doc.parse_syntax(syntax_parser)
     doc.tag_ner(ner_tagger)
     for span in doc.spans:
          span.normalize(morph_vocab)
     ner_list = []
     for span in doc.spans:
          ner_list.append((span.normal, span.type))
     return set(ner_list)

## create NER table

In [ ]:
# initial version
def get_entities(text):
     doc = Doc(text)
     doc.segment(segmenter)
     doc.tag_morph(morph_tagger)
     for token in doc.tokens:
          token.lemmatize(morph_vocab)
     doc.parse_syntax(syntax_parser)
     doc.tag_ner(ner_tagger)
     for span in doc.spans:
          span.normalize(morph_vocab)
     ner_list = []
     for span in doc.spans:
          ner_list.append((span.normal, span.type))
     return set(ner_list)

# Create an empty list to store the NERs
ner_df_list = []

# Iterate over each row in the dataframe
for index, row in tqdm(df_filtered.iterrows(), total=df_filtered.shape[0]):
    ner_list = get_entities(row['cleaned_message'])
    df_ner_temp = pd.DataFrame(ner_list, columns=['entity', 'type'])
    df_ner_temp['msg_id'] = row['msg_id']
    df_ner_temp['channel_name'] = row['channel_name']
    # Append the NERs to the list
    ner_df_list.append(df_ner_temp)

df_ner = pd.concat(ner_df_list)

100%|██████████| 5886/5886 [04:56<00:00, 19.84it/s]


## Add Narratives NLI

### local model (not openai, etc)

In [ ]:
# check available devices for M1
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

if check_environment() == 'Google Colab':
    device = "cuda" if torch.cuda.is_available() else "cpu"
else:
    device = "mps" if torch.backends.mps.is_available() else "cpu"


print(f"Using device: {device}")


model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)

Using device: cuda


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [ ]:
def get_narrat_local_model(premise, hypothesis):
    input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
    output = model(input["input_ids"].to(device))
    prediction = torch.softmax(output["logits"][0], -1).tolist()
    return prediction[0] #prob of entailment

In [ ]:
df_narr_list = []

if narr_list:
    for index, row in tqdm(df_filtered.iterrows(), total=df_filtered.shape[0]):
        for narr in narr_list:
            pred = get_narrat_local_model(row['cleaned_message'], narr)
            if pred > threshold_nli:
                channel_name = row['channel_name']
                msg_id = row['msg_id']
                df_narr_list.append((msg_id, narr, pred, channel_name))

100%|██████████| 5886/5886 [12:34<00:00,  7.81it/s]


In [ ]:
df_narr = pd.DataFrame(df_narr_list, columns=['msg_id', 'narr', 'pred', 'channel_name'])

### Narratives NLI (batch processing)

In [ ]:
## BATCH OPTION
# CUDA is explicitly called (device = 0)
# from transformers import pipeline
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device = 0)

# sequence_to_classify = test
# candidate_labels = ["politics", "economy", "entertainment", "environment"]
# output = classifier(sequence_to_classify, candidate_labels, multi_label=False, batch_size = 80)

In [ ]:
# df_narr_list = []

# if narr_list:
#     for index, row in tqdm(df_filtered.iterrows(), total=df_filtered.shape[0]):
#       pred = classifier(sequence_to_classify, narr_list, multi_label=True, batch_size = 20)


#             pred = get_narrat_local_model(row['cleaned_message'], narr)
#             if pred > threshold_nli:
#                 channel_name = row['channel_name']
#                 msg_id = row['msg_id']
#                 df_narr_list.append((msg_id, narr, pred, channel_name))

### Narratives NLI via openai

In [ ]:
# import os
# import openai

# with open('api_key.txt', 'r') as f:
#     api_key = f.readline()
# openai.api_key = api_key

# response = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {
#       "role": "system",
#       "content": f"You are given news text in Russian. You need to evaluate if the news implies the following statement:\n{hypothesis}\n?\nREPLY STRICTLY AS JSON-OBJECT WITH FOLLOWING PARAMETERS: \n{{REPLY: <TRUE OR FALSE>}}"
#     },
#     {
#       "role": "user",
#       "content": premise
#     },
#     {
#       "role": "assistant",
#       "content": "{\"REPLY\": false}"
#     }
#   ],
#   temperature=0,
#   max_tokens=50,
#   top_p=1,
#   frequency_penalty=0,
#   presence_penalty=0
# )
# response.choices[0]['message']['content']

# Clustering

In [ ]:
# Topic model
from bertopic import BERTopic
from hdbscan import HDBSCAN
# Dimension reduction
from umap import UMAP

import openai

In [ ]:
# initialize docs and embeddings
docs = df_filtered['cleaned_message']
embeddings = df_filtered.iloc[:, -384:].to_numpy()
docs.shape, embeddings.shape

((5886,), (5886, 384))

In [ ]:
umap_model = UMAP(n_neighbors=150, n_components=150, metric='cosine', low_memory=False, random_state=42)
# representation_model = KeyBERTInspired()

topic_model = BERTopic(language="russian", umap_model=umap_model, nr_topics = 'auto',\
                        calculate_probabilities=False, min_topic_size = 20
                        # , representation_model = representation_model, embedding_model = 'intfloat/multilingual-e5-large' #- KeyBERT requires embed original model
                        )
# run BERTopic clustering
topics, prob = topic_model.fit_transform(docs, embeddings = embeddings)

In [ ]:
# topics to df_filtered
df_filtered['topic'] = topics
df_filtered['prob'] = prob

In [ ]:
pd.concat(df_filtered['topic'].value_counts(normalize=False), df_filtered['topic'].value_counts(normalize=True), axis=1)

### Topic naming (openai)

In [ ]:
import openai

#load api key from local file
with open('api_key.txt', 'r') as f:
    api_key = f.readline()

openai.api_key = api_key

In [ ]:
# import time

# def ask_openai_name(topic, n_sample_news=10):
#     df_for_naming = df_filtered[(df_filtered['topic'] == topic) & (df_filtered['prob']>0.8)].sample(n_sample_news, random_state=42)
#     df_for_naming.drop_duplicates(subset=['cleaned_message'], inplace=True)
#     df_for_naming = df_for_naming.sort_values(by=['date', 'channel_name'])['cleaned_message'].apply(lambda x: x[:300])

#     for i in range(2):
#         try:
#             response = openai.ChatCompletion.create(
#                         model="gpt-3.5",
#                         messages=[
#                             {
#                             "role": "system",
#                             "content": "You are given few short texts in Russian. Generate one brief and concise description that captures the essence of all these articles. \nThe description must NOT exceed 85 characters."
#                             },
#                             {
#                             "role": "user",
#                             "content": '\n'.join(df_for_naming.to_list())
#                             }
#                         ],
#                         temperature=1,
#                         max_tokens=120,
#                         top_p=1,
#                         frequency_penalty=0,
#                         presence_penalty=0
#             )
#             name = response.choices[0]['message']['content']
#             print(f"Topic {topic} named as: {name} after {i+1} attempt(s)")
#             return name
#         except openai.error.APIConnectionError:
#             print(f"Topic {topic} failed to name after {i+1} attempt(s). Retrying...")
#             time.sleep(1)
#             continue
#     return "Error: Could not establish connection with OpenAI API after 2 attempts."


In [ ]:
# code for openai connection problems

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def ask_openai_name(topic, df_filtered, model_name="gpt-4"):
    df_for_naming = df_filtered[(df_filtered['topic'] == topic) & (df_filtered['prob']>0.8)].sample(10, random_state=42)
    df_for_naming.drop_duplicates(subset=['cleaned_message'], inplace=True)
    df_for_naming = df_for_naming.sort_values(by=['date', 'channel_name'])['cleaned_message'].apply(lambda x: x[:300])

    response = completion_with_backoff(
                model=model_name,
                messages=[
                    {
                    "role": "system",
                    "content": "You are given few short texts in Russian. Generate one brief and concise description that captures the essence of all these articles. \nThe description must NOT exceed 85 characters."
                    },
                    {
                    "role": "user",
                    "content": '\n'.join(df_for_naming.to_list())
                    }
                ],
                temperature=1,
                max_tokens=120,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
    )
    name = response.choices[0]['message']['content']
    return name

In [ ]:
topics_set = set(topics)
# remove -1
topics_set.remove(-1)
cluster_names_list = []
for topic in tqdm(topics_set):
    cluster_name = ask_openai_name(topic, df_filtered, model_name="gpt-4")
    # dataframe for cluster naming
    cluster_names_list.append((topic, cluster_name))

100%|██████████| 21/21 [02:01<00:00,  5.77s/it]


In [ ]:
df_cluster_names = pd.DataFrame(cluster_names_list, columns=['topic', 'cluster_name'])

# Output Stats

In [ ]:
# v_measure, silhouette, N of clusters, -1 size, etc.
# use code from local notebook

# OUTPUTS

## save to google drive for Dashboard

In [ ]:
# keep only necessary columns in df with texts
df_text_table = df_filtered[['channel_name', 'msg_id', 'cleaned_message', 'date', 'topic', 'views', 'number_forwards']]

In [ ]:
import os

# save dataframes to csv's as per their names
list_dataframes = [df_ner, df_narr, df_cluster_names, df_text_table, df_cross]
dataframes_names = ['df_ner', 'df_narr', 'df_cluster_names', 'df_text_table', 'df_cross']

# create backup folder if it doesn't exist
backup_path = result_path+'backups/'+request_name
if not os.path.exists(result_path+'backups/'):
    os.makedirs(result_path+'backups/')
os.makedirs(backup_path)

# save same dataframes into backup folder
for df, df_name in zip(list_dataframes, dataframes_names):
    df.to_csv(os.path.join(backup_path, f"{df_name}.csv"), index=False)

FileExistsError: ignored

# Save to google sheets

In [ ]:
import gspread
import pandas as pd
import os
from oauth2client.service_account import ServiceAccountCredentials
import sys

# Function to check env (colab or local)
def check_environment():
    if 'google.colab' in sys.modules:
        # Running in Google Colab
        return "Google Colab"

    if hasattr(sys, 'prefix'):
        if sys.prefix.startswith('/usr/local'):
            # Running in a Colab-like environment (could be Colab or another cloud environment)
            return "Colab-like environment"
        else:
            # Running in a local environment
            return "Local environment"

    # Default to local environment if checks are inconclusive
    return "Local environment"

In [ ]:
# set paths and install packages depending on the environment

if check_environment() == 'Google Colab':
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/TG news channels/'
    result_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/TG news channels/df_4_dashboard/'

else:
    data_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/TG news channels/'
    result_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/TG news channels/df_4_dashboard/'

In [ ]:
# get dataframes
dataframes_names = ['df_ner', 'df_narr', 'df_cluster_names', 'df_text_table', 'df_cross']

# load dataframes as per their names
for df_name in dataframes_names:
    df = pd.read_csv(os.path.join(result_path, f"{df_name}.csv"))
    globals()[df_name] = df

In [ ]:
# save dataframes to google sheets
# define the scope
scope = ['https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name(result_path+'newsnarratives-fa6127a3d237.json', scope)

# authorize the clientsheet
client = gspread.authorize(creds)

# get the instance of the Spreadsheet
sheet = client.open('df_narr')

In [ ]:
sheet_instance = sheet.get_worksheet(0)

# clear the sheet
sheet_instance.clear()

# Export the DataFrame to Google Sheets
sheet_instance.update([df_narr.columns.values.tolist()] + df_narr.values.tolist())

{'spreadsheetId': '14TqncRCevwDiAkndLAzwtsWRz1XFIvy8Rm2gJePwEgM',
 'updatedRange': 'df_narr!A1:D733',
 'updatedRows': 733,
 'updatedColumns': 4,
 'updatedCells': 2932}

In [ ]:
# update all google sheets with respective dataframes
for df_name in dataframes_names:
    print(f"Updating {df_name}...")
    df = globals()[df_name]
    sheet = client.open(df_name)
    sheet_instance = sheet.get_worksheet(0)
    sheet_instance.clear()
    df = df.fillna('')
    sheet_instance.update([df.columns.values.tolist()] + df.values.tolist())
    print(f"{df_name} updated successfully!")

Updating df_ner...
df_ner updated successfully!
Updating df_narr...
df_narr updated successfully!
Updating df_cluster_names...
df_cluster_names updated successfully!
Updating df_text_table...
df_text_table updated successfully!
Updating df_cross...
df_cross updated successfully!
